In [7]:
import subprocess

diff = subprocess.check_output(["git", "diff", "origin/main...HEAD"]).decode("utf-8")

In [8]:
print(diff)

diff --git a/README.md b/README.md
index 55ecab9..46a422e 100644
--- a/README.md
+++ b/README.md
@@ -1,6 +1,7 @@
-```markdown
 # 🌟 CartAI 🌟
 
+[![Build Status](https://github.com/ContrastoAI/cartai/actions/workflows/pre-commit.yaml/badge.svg)](https://github.com/ContrastoAI/cartai/actions)
+
 ## 📝 Description
 
 Crafting intelligent E2E documentation for trustworthy AI.
diff --git a/cartai/cli/commands/__init__.py b/cartai/cli/commands/__init__.py
new file mode 100644
index 0000000..0a02c7d
--- /dev/null
+++ b/cartai/cli/commands/__init__.py
@@ -0,0 +1,3 @@
+"""
+CartAI CLI commands package
+"""
diff --git a/cartai/cli/commands/pr_diff.py b/cartai/cli/commands/pr_diff.py
new file mode 100644
index 0000000..0130706
--- /dev/null
+++ b/cartai/cli/commands/pr_diff.py
@@ -0,0 +1,41 @@
+"""
+PR diff analysis command implementation
+"""
+
+import typer
+from rich.console import Console
+
+console = Console()
+
+
+def pr_diff_command(
+    pull_request: int = typer.Option(..., help="Pull requ

In [9]:
from cartai.llm_agents.documenter import AIDocumenter

documenter = AIDocumenter()
documenter._load_template("pr_diff.jinja").render(
    {
        "pr_title": "Add new feature",
        "description": "Add a new feature to the project",
        "diff": diff,
    },
)

'You are a helpful code reviewer. Your task is to analyze the following pull request diff and provide a clear, concise summary.\n\nThe diff is from a pull request: Add new feature\n\n\nPull Request Description:\nAdd a new feature to the project\n\n\n\n\nDiff content:\ndiff --git a/README.md b/README.md\nindex 55ecab9..46a422e 100644\n--- a/README.md\n+++ b/README.md\n@@ -1,6 +1,7 @@\n-```markdown\n # 🌟 CartAI 🌟\n \n+[![Build Status](https://github.com/ContrastoAI/cartai/actions/workflows/pre-commit.yaml/badge.svg)](https://github.com/ContrastoAI/cartai/actions)\n+\n ## 📝 Description\n \n Crafting intelligent E2E documentation for trustworthy AI.\ndiff --git a/cartai/cli/commands/__init__.py b/cartai/cli/commands/__init__.py\nnew file mode 100644\nindex 0000000..0a02c7d\n--- /dev/null\n+++ b/cartai/cli/commands/__init__.py\n@@ -0,0 +1,3 @@\n+"""\n+CartAI CLI commands package\n+"""\ndiff --git a/cartai/cli/commands/pr_diff.py b/cartai/cli/commands/pr_diff.py\nnew file mode 100644\nindex 

In [8]:
llm_output = documenter.generate(
    "pr_diff.jinja",
    {
        "pr_title": "Add new feature",
        "description": "Add a new feature to the project",
        "diff": diff,
    },
)

In [10]:
print(llm_output)

### Summary of Main Changes:
This pull request introduces a new template file (`pr_diff.jinja`) for generating summaries of pull request diffs in the project. The template is designed to help code reviewers provide concise feedback by outlining the main changes, files affected, and any potential concerns.

### Key Modifications by Category:
- **Features**: 
  - Added a new Jinja template (`pr_diff.jinja`) for summarizing pull request diffs.
  - The template includes placeholders for pull request titles, descriptions, files changed, and a structured format for reviewer feedback.

### Potential Concerns or Areas That Need Attention:
- Ensure that the template is integrated correctly with existing workflows for pull requests.
- Verify that the generated summaries are clear and useful for reviewers, potentially requiring user feedback for improvements.
- Consider edge cases where descriptions or files might be absent and how the template handles those scenarios.


In [14]:
from cartai.core.code_parser import ProjectParser


parser = ProjectParser(
    include_basic_entities=True,
)

project_structure_summary = parser.get_summary("../")
project_structure_summary

'📁 ../\n  📄 .env.example\n  📁 .github/\n    📁 workflows/\n      📄 pre-commit.yaml\n  📄 .pre-commit-config.yaml\n  📄 .python-version\n  📁 cartai/\n    📄 __init__.py\n    📁 adapters/\n    📁 cli/\n      📁 commands/\n        📄 __init__.py\n        📄 pr_diff.py\n        📄 readme.py\n      📄 main.py\n    📁 core/\n      📄 __init__.py\n      📄 code_parser.py\n    📁 lineage/\n    📁 llm_agents/\n      📄 __init__.py\n      📄 documenter.py\n      📁 templates/\n        📄 pr_diff.jinja\n        📄 readme.jinja\n      📄 utils.py\n  📄 LICENSE\n  📁 llm_agents/\n    📁 templates/\n  📄 Makefile\n  📄 pyproject.toml\n  📄 README.md\n  📄 uv.lock'

In [15]:
llm_output_w_code = documenter.generate(
    "pr_diff.jinja",
    {
        "pr_title": "Add new feature",
        "description": "Add a new feature to the project",
        "diff": diff,
        "project_code": project_structure_summary,
    },
)

In [17]:
print(llm_output_w_code)

### Pull Request Summary

1. **Brief Summary of the Main Changes:**
   This pull request introduces a new Jinja template file (`pr_diff.jinja`) to the `cartai/llm_agents/templates` directory, which is designed to aid in generating summaries for pull request diffs. The template includes sections for the pull request title, description, files changed, and diff content, as well as prompts for code reviewers to provide feedback.

2. **Key Modifications by Category:**
   - **Feature Addition:** A new template file (`pr_diff.jinja`) has been added to streamline the code review process by providing a structured format for summarizing pull requests.

3. **Potential Concerns or Areas That Need Attention:**
   - **Template Usage:** Ensure that the new template is integrated properly into the existing workflow for pull request reviews. It may be beneficial to provide documentation or examples on how to use this template effectively.
   - **Testing:** Consider adding tests to verify that the templ

# Test the pr_diff func

In [1]:
import os
import requests

from cartai.cli.commands.pr_diff import pr_diff_command


pr_number = 9

headers = {
    "Authorization": f"token {os.getenv('GITHUB_TOKEN')}",
    "Accept": "application/vnd.github.v3+json",
}
pr_url = (
    f"https://api.github.com/repos/{os.getenv('GITHUB_REPOSITORY')}/pulls/{pr_number}"
)
pr_data = requests.get(pr_url, headers=headers).json()
original_body = pr_data.get("body", "") or ""

In [6]:
original_body

''

In [7]:
import subprocess

from cartai.cli.commands.pr_diff import _is_excluded

raw_diff = subprocess.check_output(
    ["git", "diff", "--name-only", "origin/main...HEAD"]
).decode("utf-8")
files = [
    f.strip()
    for f in raw_diff.splitlines()
    if f.strip() and not _is_excluded(f.strip())
]
files

['cartai/cli/commands/__init__.py',
 'cartai/cli/commands/pr_diff.py',
 'cartai/cli/commands/readme.py',
 'cartai/cli/main.py',
 'cartai/llm_agents/templates/pr_diff.jinja',
 'examples/02_pr_diff.ipynb']

In [28]:
# Check if files have any changes at all
print("Status of files:")
for file in files:
    status = subprocess.check_output(["git", "status", "--porcelain", file]).decode(
        "utf-8"
    )
    print(f"{file}: {status or 'no changes'}")

Status of files:
cartai/cli/commands/__init__.py: no changes
cartai/cli/commands/pr_diff.py: no changes
cartai/cli/commands/readme.py: no changes
cartai/cli/main.py: no changes
cartai/llm_agents/templates/pr_diff.jinja: no changes
examples/02_pr_diff.ipynb: no changes


In [5]:
pr_diff_command(pull_request="9", repo=os.getenv("GITHUB_REPOSITORY"))

PR description updated with summary.
